# Denoising with BM3D

The code below demonstrates the denoising performance on an example image.

In [1]:
#from __future__ import print_function
#import matplotlib.pyplot as plt
#%matplotlib notebook

import os

from os import *
from os.path import *

import warnings
warnings.filterwarnings('ignore')

#from include import *
from PIL import Image
import PIL

import numpy as np
import pybm3d

##  Load images

In [2]:
def pil_to_np(img_PIL):
    '''Converts image in PIL format to np.array.
    
    From W x H x C [0...255] to C x W x H [0..1]
    '''
    ar = np.array(img_PIL)

    if len(ar.shape) == 3:
        ar = ar.transpose(2,0,1)
    else:
        ar = ar[None, ...]

    return ar.astype(np.float32) / 255.

def load_and_crop(imgname,target_width=512,target_height=512):
    '''
    imgname: string of image location
    load an image, and center-crop if the image is large enough, else return none
    '''
    img = Image.open(imgname)
    width, height = img.size
    if width <= target_width or height <= target_height:
        return None	
    
    left = (width - target_width)/2
    top = (height - target_height)/2
    right = (width + target_width)/2
    bottom = (height + target_height)/2
    
    return img.crop((left, top, right, bottom))

def get_imgnet_imgs(path = './imgs/'):
    siz = 512
    imgs = []
    imgnames = [f for f in listdir(path) if isfile(join(path, f))]   
    for imgname in imgnames:
        # prepare and select image
        imgname = path + imgname

        img = load_and_crop(imgname,target_width=512,target_height=512)
        if img is None: # then the image could not be croped to 512x512
            continue
             
        img_np = pil_to_np(img)

        if img_np.shape[0] != 3: # we only want to consider color images
            continue
        imgs += [img_np]
    print("got ", len(imgs), " images")
    return imgs

imgs = get_imgnet_imgs()

got  100  images


In [1]:
## to greyscale
def rgb2gray(rgb):
    return 0.2989*rgb[0] + 0.5870*rgb[1] + 0.1140*rgb[2]

#gimg = np.array([rgb2gray(imgs[0])])
#gimg = gimg.transpose(1,2,0)

In [11]:
def get_noisy_img(img_np,sig=30,noise_same = False):
    sigma = sig/255.
    if noise_same: # add the same noise in each channel
        noise1 = np.random.normal(scale=sigma, size=img_np.shape[:2])
        noise = np.zeros(img_np.shape)
        noise[:,:,0] = noise1
        noise[:,:,1] = noise1
        noise[:,:,2] = noise1
    else: # add independent noise in each channel
        noise = np.random.normal(scale=sigma, size=img_np.shape)

    img_noisy_np = np.clip( img_np + noise , 0, 1).astype(np.float32)
    #img_noisy_var = np_to_var(img_noisy_np).type(dtype)
    return img_noisy_np #,img_noisy_var

In [6]:
def psnr(x_hat,x_true,maxv=1.):
    x_hat = x_hat.flatten()
    x_true = x_true.flatten()
    mse=np.mean(np.square(x_hat-x_true))
    psnr_ = 10.*np.log(maxv**2/mse)/np.log(10.)
    return psnr_

## Denoise noisy image

In [12]:
sig = 25.0
noise_same = True

In [13]:
psnrs = []
for img in imgs:
    # get noisy img
    
    # make grayscale
    #gimg = np.array([rgb2gray(imgs[0])])
    #img = gimg.transpose(1,2,0)
    img = img.transpose(1,2,0)
    
    img_noisy_np = get_noisy_img(img,sig=sig,noise_same=noise_same)
    output_depth = img.shape[0] 
    print(img.shape, np.max(img), np.min(img))
    
    # denoise
    sigma = sig/255.
    out_img_np = pybm3d.bm3d.bm3d(img_noisy_np, sigma)
    #img = img.transpose(2,1,0)
    
    print("Noise PSNR: ",psnr(img,img_noisy_np) )
    print("Recov PSNR: ",psnr(img,out_img_np) )
    psnrv = psnr(img,out_img_np)
    psnrs.append(psnrv)
print(np.mean(psnrs))
print(np.mean(psnrs))

(512, 512, 3) 1.0 0.0
Noise PSNR:  20.212181200072777
Recov PSNR:  23.63727431083024
(512, 512, 3) 0.972549 0.0
Noise PSNR:  20.504885441401665
Recov PSNR:  23.940315558364137
(512, 512, 3) 1.0 0.0
Noise PSNR:  20.301638870226878
Recov PSNR:  23.696178870942163
(512, 512, 3) 1.0 0.0
Noise PSNR:  20.768846524713283
Recov PSNR:  28.750837924533748
(512, 512, 3) 1.0 0.0
Noise PSNR:  20.594251079013212
Recov PSNR:  25.36551000419403
(512, 512, 3) 1.0 0.0
Noise PSNR:  20.464093268981927
Recov PSNR:  23.997677276501193
(512, 512, 3) 1.0 0.0
Noise PSNR:  20.57955428958745
Recov PSNR:  24.469921477217806
(512, 512, 3) 1.0 0.0
Noise PSNR:  20.54248374191298
Recov PSNR:  26.705066584818994
(512, 512, 3) 1.0 0.0
Noise PSNR:  20.59146513458877
Recov PSNR:  24.70622667488254
(512, 512, 3) 0.99215686 0.0
Noise PSNR:  20.480707008180083
Recov PSNR:  25.4927768415609
(512, 512, 3) 1.0 0.0
Noise PSNR:  20.56513601853171
Recov PSNR:  27.63120525150335
(512, 512, 3) 1.0 0.0
Noise PSNR:  20.85872056313074